In [1]:
import pandas as pd
import os
import sys
from dotenv import load_dotenv

# Load environment variables from the .env file
load_dotenv()

# Access the API key
PINECONE_API_KEY = os.getenv("PINECONE_API_KEY")
EMBED_DIM = 768

# Data

In [2]:
df = pd.read_csv('supports.csv')

df.head()

,Unnamed: 0,title,link,content
0,0,"<div class=""text_logo"">Tài khoản</div>",https://hotro.hasaki.vn/tai-khoan.html,Tài khoản Hasaki | Hasaki.vn Gửi yêu cầu | Đăn...
1,1,"<div class=""text_logo"">Đặt hàng</div>",https://hotro.hasaki.vn/dat-hang-tai-hasaki.html,Đặt hàng tại Hasaki | Hasaki.vn Gửi yêu cầu | ...
2,2,"<div class=""text_logo"">Quy cách đóng gói</div>",https://hotro.hasaki.vn/quy-cach-dong-goi.html,Quy cách đóng gói hàng tại Hasaki | Hasaki.vn ...
3,3,"<div class=""text_logo"">Vận chuyển 2H</div>",https://hotro.hasaki.vn/van-chuyen-2h.html,Vận chuyển 2H tại Hasaki | Hasaki.vn Gửi yêu c...
4,4,"<div class=""text_logo"">Phí vận chuyển</div>",https://hotro.hasaki.vn/phi-van-chuyen.html,Phí vận chuyển tại Hasaki | Hasaki.vn Gửi yêu ...


In [3]:
import re

df['title'] = df['title'].apply(lambda title: re.sub(r'<[^>]+>', '', title))

In [4]:
len(df)

26

# Chunking

In [5]:
import time
from awan_llm_api import AwanLLMClient, Role
from awan_llm_api.completions import ChatCompletions
import sys
import os
from dotenv import load_dotenv

load_dotenv()

AWANLLM_API_KEY = os.getenv('AWAN_API_KEY')
MODEL = os.getenv('MODEL_NAME_LARGE')

class AwanAPI:
    def __init__(self, api_key=AWANLLM_API_KEY, model_name=MODEL):
        self.api_token = api_key
        self.model_name = MODEL
        self.history = []
        self.last_request_time = None  # Initialize the last request time

        if self.api_token and self.model_name:
            self.client = AwanLLMClient(self.api_token)
            self.chat = ChatCompletions(self.model_name)

    def add_to_history(self, role, message):
        self.history.append({'role': role, 'message': message})

    def get_response(self, prompt, customer_query, mode="text"):
        self.add_to_history(Role.SYSTEM, prompt)  
        self.add_to_history(Role.USER, customer_query)

        # Check time elapsed since last request to stay under rate limit (20 requests/min)
        if self.last_request_time:
            time_since_last_request = time.time() - self.last_request_time
            min_interval_between_requests = 60 / 20  # 3 seconds for 20 req/min

            if time_since_last_request < min_interval_between_requests:
                time_to_wait = min_interval_between_requests - time_since_last_request
                print(f"Rate limit in effect. Waiting for {time_to_wait:.2f} seconds.")
                time.sleep(time_to_wait)

        # Process the request and store the time
        for entry in self.history:
            self.chat.add_message(entry['role'], entry['message'])

        response = self.client.chat_completion(self.chat)
        self.last_request_time = time.time()  # Update the last request time
        self.add_to_history(Role.ASSISTANT, response)

        if mode == "json":
            return {
                "response": response,
                "history": self.history
            }
        return response  # Default to returning text if no mode is specified


In [6]:
def chunk(text):
    chunk_llm = AwanAPI()

    prompt = """Giúp tôi xử lý query tôi đưa vào. Bắt buộc xóa các dấu nháy đơn ' ' và nháy kép " " “ ” trước khi xử lý.
                Đầu tiên hãy xóa bỏ phần ở trong các dấu | | và những thông tin quảng cáo ở cuối đoạn
                Sau đó giúp tôi chunking nhỏ đoạn query thành những đoạn vẫn có đầy đủ ý nghĩa. Mỗi đoạn nhỏ khoảng 200 từ phải có đủ ý nghĩa của 1 đoạn.            
                Đưa ra câu trả lời theo dạng 1 list, chỉ gồm các đoạn được chunking
                Không trả thêm bất kỳ thứ gì ngoài các đoạn được chunking kể cả lời dẫn .
                Mỗi đoạn chunking là 1 phần tử trong list.
                Trả về theo đúng định dạng sau: Nếu chunk được thành 2 đoạn thì trả về "['đoạn chunk 1', 'đoạn chunk 2']"
            """
    
    chat_response = chunk_llm.get_response(prompt, text)
    
    return chat_response.get('choices')[0]['message']['content']

In [7]:
chunks = []

In [12]:
for i in range(15, 26):
    chunks.append(chunk(df['content'][i]))

In [22]:
len(chunks)

26

In [24]:
import ast

list_chunk = [None] * 26
for i in range(0,26):
    if chunks[i][0] == "[":
        list_chunk[i] = ast.literal_eval(chunks[i])
    else:
        list_chunk[i] = chunks[i]

In [63]:
df['chunked_content'] = list_chunk

In [67]:
df.head()

,Unnamed: 0,title,link,content,chunked_content
0,0,Tài khoản,https://hotro.hasaki.vn/tai-khoan.html,Tài khoản Hasaki | Hasaki.vn Gửi yêu cầu | Đăn...,[Đăng ký thành viên tại Hasaki như thế nào? Qu...
1,1,Đặt hàng,https://hotro.hasaki.vn/dat-hang-tai-hasaki.html,Đặt hàng tại Hasaki | Hasaki.vn Gửi yêu cầu | ...,"[Tài khoản, Đặt hàng Quy cách đóng gói Vận chu..."
2,2,Quy cách đóng gói,https://hotro.hasaki.vn/quy-cach-dong-goi.html,Quy cách đóng gói hàng tại Hasaki | Hasaki.vn ...,[Quy cách đóng gói hàng tại Hasaki | Hasaki.vn...
3,3,Vận chuyển 2H,https://hotro.hasaki.vn/van-chuyen-2h.html,Vận chuyển 2H tại Hasaki | Hasaki.vn Gửi yêu c...,[Vận chuyển 2H tại Hasaki | Hasaki.vn Tài khoả...
4,4,Phí vận chuyển,https://hotro.hasaki.vn/phi-van-chuyen.html,Phí vận chuyển tại Hasaki | Hasaki.vn Gửi yêu ...,[Hasaki miễn phí vận chuyển tại các tỉnh/ thàn...


In [65]:
def to_list(text):
    if isinstance(text, str):
        return [text]
    else:
        return text
    
df['chunked_content'] = df['chunked_content'].apply(lambda text: to_list(text))

In [73]:
df['chunked_content'] = df.apply(lambda row: [row['title']] + row['chunked_content'], axis=1)


In [75]:
df_ = df.explode('chunked_content', ignore_index=True)

In [83]:
df_.head()

,Unnamed: 0,title,link,content,chunked_content
0,0,Tài khoản,https://hotro.hasaki.vn/tai-khoan.html,Tài khoản Hasaki | Hasaki.vn Gửi yêu cầu | Đăn...,Tài khoản
1,0,Tài khoản,https://hotro.hasaki.vn/tai-khoan.html,Tài khoản Hasaki | Hasaki.vn Gửi yêu cầu | Đăn...,Đăng ký thành viên tại Hasaki như thế nào? Quý...
2,0,Tài khoản,https://hotro.hasaki.vn/tai-khoan.html,Tài khoản Hasaki | Hasaki.vn Gửi yêu cầu | Đăn...,Tại sao tôi không thể đăng nhập vào tài khoản ...
3,0,Tài khoản,https://hotro.hasaki.vn/tai-khoan.html,Tài khoản Hasaki | Hasaki.vn Gửi yêu cầu | Đăn...,Tôi muốn thay đổi thông tin tài khoản thành vi...
4,1,Đặt hàng,https://hotro.hasaki.vn/dat-hang-tai-hasaki.html,Đặt hàng tại Hasaki | Hasaki.vn Gửi yêu cầu | ...,Đặt hàng


# Connect Pinecone

In [80]:
from pinecone_connection import PineConeDB

hasaki_index = PineConeDB(PINECONE_API_KEY).pc.Index('hasaki-index')

In [81]:
index_stats = hasaki_index.describe_index_stats()

print("Namespaces:", index_stats['namespaces'].keys())

Namespaces: dict_keys(['product-pname-namespace', 'category-namespace', 'product-desc-namespace', 'campaign-namespace', 'support-namespace'])


# Up Sert

In [84]:
from pinecone_connection import create_vector_emb

df_['vector'] = df_['chunked_content'].apply(lambda x: create_vector_emb(x))

In [85]:
df_.head()

,Unnamed: 0,title,link,content,chunked_content,vector
0,0,Tài khoản,https://hotro.hasaki.vn/tai-khoan.html,Tài khoản Hasaki | Hasaki.vn Gửi yêu cầu | Đăn...,Tài khoản,"[-0.027002433314919472, -0.0013075470924377441..."
1,0,Tài khoản,https://hotro.hasaki.vn/tai-khoan.html,Tài khoản Hasaki | Hasaki.vn Gửi yêu cầu | Đăn...,Đăng ký thành viên tại Hasaki như thế nào? Quý...,"[-0.19404232501983643, 0.2964792847633362, -0...."
2,0,Tài khoản,https://hotro.hasaki.vn/tai-khoan.html,Tài khoản Hasaki | Hasaki.vn Gửi yêu cầu | Đăn...,Tại sao tôi không thể đăng nhập vào tài khoản ...,"[-0.15494681894779205, 0.3523898422718048, -0...."
3,0,Tài khoản,https://hotro.hasaki.vn/tai-khoan.html,Tài khoản Hasaki | Hasaki.vn Gửi yêu cầu | Đăn...,Tôi muốn thay đổi thông tin tài khoản thành vi...,"[0.02982049435377121, 0.1364598125219345, -0.2..."
4,1,Đặt hàng,https://hotro.hasaki.vn/dat-hang-tai-hasaki.html,Đặt hàng tại Hasaki | Hasaki.vn Gửi yêu cầu | ...,Đặt hàng,"[0.19458062946796417, 0.7691003680229187, -0.3..."


In [93]:
upsert_data = []
for idx, row in df_.iterrows():
    vector = row['vector']
    metadata = {
        "title": row['title'],
        "content": row['content'],
        "link": row['link']
    }
    upsert_data.append((str(idx), vector, metadata))

In [94]:
hasaki_index.upsert(vectors=upsert_data, namespace='support-namespace')

{'upserted_count': 121}